In [1]:
import pandas as pd
import pandas_gbq
import plotly.express as px
import sys, requests
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
query = """
(SELECT 
*, "Regular Season" as Category
FROM `perceptive-ivy-290216.nba_api_individual_matchups.all_time_reg_season` 
)
UNION ALL
(SELECT 
*, "Playoffs" as Category
FROM `perceptive-ivy-290216.nba_api_individual_matchups.all_time_playoffs`
)
"""
project_id = "perceptive-ivy-290216"
matchups_totals = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [15]:
query = """
SELECT PLAYER_NAME MATCHUP_NAME, TEAM_ABBREVIATION, SEASON FROM `perceptive-ivy-290216.player_team_info.player_bios_regs_totals`
"""
project_id = "perceptive-ivy-290216"
player_team = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [4]:
query = """
SELECT GAME_ID, GAME_DATE, MATCHUP	
FROM `perceptive-ivy-290216.nba_game_player_team_ids.game_ids_reg_season`
WHERE MATCHUP LIKE "%@%"
UNION ALL
SELECT GAME_ID,GAME_DATE, MATCHUP
FROM `perceptive-ivy-290216.nba_game_player_team_ids.game_ids_playoffs`
WHERE MATCHUP LIKE "%@%"
"""
project_id = "perceptive-ivy-290216"
game_ids = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [19]:
def team_color(team):
    if team =="ATL":
        return "#C8102E"
    elif team =="DAL":
        return '#0050B5'
    elif team =="MEM":
        return '#0C2340'
    elif team =="MIA":
        return '#862633'
    elif team =="DEN":
        return '#1d428a'
    elif team =="DET":
        return '#1D4289'
    elif team =="GSW":
        return '#1D4289'
    elif team =="HOU":
        return '#BA0C2F'
    elif team =="IND":
        return '#041E42'
    elif team =="LAC":
        return '#0C2340'
    elif team =="LAL":
        return '#6a00ed'
    elif team =="POR":
        return '#C8102E'
    elif team =="PHX":
        return '#e56020'
    elif team =="PHI":
        return '#1D4289'
    elif team =="ORL":
        return '#0057B7'
    elif team =="OKC":
        return '#0072CE'
    elif team =="NYK":
        return '#FF8200'
    elif team =="NOP":
        return '#85714d'
    elif team =="MIN":
        return '#236192'
    elif team =="MIL":
        return '#2C5234'
    elif team =="BKN":
        return '#707372'
    elif team =="BOS":
        return '#007A33'
    elif team =="SEA":
        return '#00653A'
    elif team =="CHA":
        return '#00778B'
    elif team =="BUF":
        return '#418FDE'
    elif team =="CHI":
        return '#BA0C2F'       
    elif team =="CLE":
        return '#6F263D'
    elif team =="WAS":
        return '#C8102E'
    elif team =="UTA":
        return '#0C2340' 
    elif team =="TOR":
        return '#C8102E'
    elif team =="SAS":
        return '#9EA2A2'
    elif team =="SAC":
        return '#965ddd'                  
    
player_team['color'] = player_team['TEAM_ABBREVIATION'].apply(team_color)

In [20]:
#REGULAR SEASON

In [21]:
#TOTALS

In [22]:
year="2023-24"

In [24]:
#Combine player team data with player aggregated data so that we can color based on team
matchup_totals_year=matchups_totals[matchups_totals["Season"]==year].merge(player_team[player_team["SEASON"]==year], left_on=["MATCHUP_NAME","Season"], right_on=["MATCHUP_NAME","SEASON"])
matchup_totals_year=matchup_totals_year.merge(game_ids, left_on=["GAME_ID"], right_on=["GAME_ID"])
player_color_dict_shot_dist=dict(zip(matchup_totals_year['MATCHUP_NAME'], matchup_totals_year['color']))
team_color_dict_shot_dist=dict(zip(matchup_totals_year['TEAM_ABBREVIATION'], matchup_totals_year['color']))
matchup_totals_year.tail()

,GAME_ID,HOME_TEAM,AWAY_TEAM,TEAM,PLAYER_NAME,POSITION,MATCHUP_NAME,helpBlocks,helpFieldGoalsAttempted,helpFieldGoalsMade,helpFieldGoalsPercentage,matchupAssists,matchupBlocks,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupFieldGoalsPercentage,matchupFreeThrowsAttempted,matchupFreeThrowsMade,matchupMinutes,matchupMinutesSort,matchupPotentialAssists,matchupThreePointersAttempted,matchupThreePointersMade,matchupThreePointersPercentage,matchupTurnovers,partialPossessions,percentageDefenderTotalTime,percentageOffensiveTotalTime,percentageTotalTimeBothOn,playerPoints,shootingFouls,switchesOn,teamPoints,Season,Category,TEAM_ABBREVIATION,SEASON,color,GAME_DATE,MATCHUP
244230,0022300712,Denver Nuggets,Portland Trail Blazers,Portland Trail Blazers,Toumani Camara,,Peyton Watson,0,0,0,0.0,0,0,3,2,0.667,0,0,1:19,78,0,1,1,1.000,1,4.8,0.108,0.146,0.164,5,0,0,11,2023-24,Regular Season,DEN,2023-24,#1d428a,2024-02-04,POR @ DEN
244231,0022300348,Portland Trail Blazers,Golden State Warriors,Portland Trail Blazers,Jerami Grant,F,Dario Saric,0,0,0,0.0,2,0,4,2,0.500,0,0,1:01,60,0,3,1,0.333,1,3.9,0.186,0.068,0.212,5,0,0,14,2023-24,Regular Season,GSW,2023-24,#1D4289,2023-12-17,GSW @ POR
244232,0022300986,Chicago Bulls,Portland Trail Blazers,Portland Trail Blazers,Anfernee Simons,G,Onuralp Bitim,0,0,0,0.0,0,0,3,1,0.333,2,2,0:40,40,0,3,1,0.333,0,5.1,0.069,0.045,0.087,5,1,0,10,2023-24,Regular Season,CHI,2023-24,#BA0C2F,2024-03-18,POR @ CHI
244233,0022301183,Portland Trail Blazers,Houston Rockets,Portland Trail Blazers,Dalano Banton,,Dillon Brooks,0,0,0,0.0,1,0,3,1,0.333,2,2,1:23,83,0,3,1,0.333,0,8.5,0.116,0.092,0.165,5,0,0,11,2023-24,Regular Season,HOU,2023-24,#BA0C2F,2024-04-12,HOU @ POR
244234,0022300881,Minnesota Timberwolves,Portland Trail Blazers,Portland Trail Blazers,Dalano Banton,F,Anthony Edwards,0,0,0,0.0,0,0,5,3,0.600,0,0,3:30,209,0,1,0,0.000,0,14.7,0.238,0.249,0.318,6,0,0,13,2023-24,Regular Season,MIN,2023-24,#236192,2024-03-04,POR @ MIN


In [ ]:
#By Individual Games

In [8]:
game_id="0022300712"

In [25]:
#Filter to regular season
matchup_season=matchup_totals_year[(matchup_totals_year["Category"]=='Regular Season')&(matchup_totals_year["GAME_ID"]==game_id)]
#Create a player and team dict for color for future coloring

In [80]:
matchup=matchup_season['MATCHUP'].iloc[0]
date=matchup_season['GAME_DATE'].iloc[0]

In [27]:
category="matchupFreeThrowsAttempted"

# category=['helpBlocks', 'helpFieldGoalsAttempted',
#        'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
#        'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
#        'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
#        'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
#        'matchupPotentialAssists', 'matchupThreePointersAttempted',
#        'matchupThreePointersMade', 'matchupThreePointersPercentage',
#        'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
#        'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
#        'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']

In [81]:
fig_matchup_season = px.bar(
  matchup_season[matchup_season[category]!=0].sort_values(by=category, ascending=False),
  x="MATCHUP_NAME", 
  y=category,
  color="MATCHUP_NAME",
  color_discrete_map=player_color_dict_shot_dist,     
  template='plotly_white',
  title="<b>Matchup {} For The {} Game on {}</b>".format(category,matchup,date),
  hover_data=['GAME_DATE','MATCHUP','TEAM', 'PLAYER_NAME',
       'MATCHUP_NAME', 'helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints',
       'Category', 'SEASON', ], 
  height=800, 
  width=1200,
  )
fig_matchup_season.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_matchup_season

In [ ]:
fig_matchup_season.write_html("Matchup {} For The {} Game on {}".format(category,matchup,date),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOT

In [ ]:
category_multi=['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']
for i in category_multi:
       fig_matchup_season_multi = px.bar(
       matchup_season[matchup_season[i]!=0].sort_values(by=i, ascending=False),
       x="MATCHUP_NAME", 
       y=i,
       color="MATCHUP_NAME",
       color_discrete_map=player_color_dict_shot_dist,     
       template='plotly_white',
       title="<b>Matchup {} For The {} Game on {}</b>".format(i,matchup,date),
       hover_data=['GAME_DATE','MATCHUP','TEAM', 'PLAYER_NAME',
              'MATCHUP_NAME', 'helpBlocks', 'helpFieldGoalsAttempted',
              'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
              'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
              'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
              'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
              'matchupPotentialAssists', 'matchupThreePointersAttempted',
              'matchupThreePointersMade', 'matchupThreePointersPercentage',
              'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
              'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
              'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints',
              'Category', 'SEASON', ], 
       height=800, 
       width=1200,
       )
       fig_matchup_season_multi.update_layout(
       margin=dict(l=5, r=5, t=40, b=5),
       showlegend=False,
       title_x=0.5,
       hoverlabel=dict(
              bgcolor="white",
              font_size=16,
              font_family="PT Sans Narrow"
       ),
       yaxis = dict(tickfont = dict(size=15)),
       xaxis = dict(tickfont = dict(size=15)),
       font=dict(
              family="PT Sans Narrow",
              size=14,
              color="Black"
       ),
       title_font_family="PT Sans Narrow"
       )
       fig_matchup_season_multi
       fig_matchup_season_multi.write_html("Matchup {} For The {} Game on {}".format(i,matchup,date),full_html=False, include_plotlyjs='cdn')

In [ ]:
# For Total Season

In [42]:
matchup_season_full=matchup_totals_year[(matchup_totals_year["Category"]=='Regular Season')]

In [43]:
matchup_season_full_totals=pd.DataFrame(matchup_season_full.groupby(by=['MATCHUP_NAME','PLAYER_NAME',"SEASON"])[['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
        'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']].sum()).reset_index()
matchup_season_full_totals["matchupFieldGoalsPercentage"]=matchup_season_full_totals["matchupFieldGoalsMade"]/matchup_season_full_totals["matchupFieldGoalsAttempted"]
matchup_season_full_totals["matchupThreePointersPercentage"]=matchup_season_full_totals["matchupThreePointersMade"]/matchup_season_full_totals["matchupThreePointersAttempted"]
matchup_season_full_totals["matchupFreeThrowsPercentage"]=matchup_season_full_totals["matchupFreeThrowsMade"]/matchup_season_full_totals["matchupFreeThrowsAttempted"]
matchup_season_full_totals=matchup_season_full_totals.fillna(0)

In [44]:
matchup_season_full_totals.head()

,MATCHUP_NAME,PLAYER_NAME,SEASON,helpBlocks,helpFieldGoalsAttempted,helpFieldGoalsMade,matchupAssists,matchupBlocks,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupFreeThrowsAttempted,matchupFreeThrowsMade,matchupMinutesSort,matchupPotentialAssists,matchupThreePointersAttempted,matchupThreePointersMade,matchupTurnovers,partialPossessions,playerPoints,shootingFouls,switchesOn,teamPoints,matchupFieldGoalsPercentage,matchupThreePointersPercentage,matchupFreeThrowsPercentage
0,A.J. Lawson,Aaron Holiday,2023-24,0,0,0,0,0,1,0,0,0,94,0,1,0,1,11.1,0,0,0,3,0.0,0.0,NaN
1,A.J. Lawson,Aaron Wiggins,2023-24,0,0,0,0,0,1,0,0,0,86,0,0,0,0,6.5,0,0,0,4,0.0,NaN,NaN
2,A.J. Lawson,Alperen Sengun,2023-24,0,0,0,0,0,1,0,0,0,5,0,0,0,0,0.4,0,0,0,0,0.0,NaN,NaN
3,A.J. Lawson,Amen Thompson,2023-24,0,0,0,1,0,0,0,0,0,136,0,0,0,1,12.0,0,0,0,14,NaN,NaN,NaN
4,A.J. Lawson,Andre Drummond,2023-24,0,0,0,0,0,1,0,0,0,3,0,0,0,0,0.2,0,0,0,0,0.0,NaN,NaN


In [45]:
category_full_season="matchupFreeThrowsAttempted"

# category=['helpBlocks', 'helpFieldGoalsAttempted',
#        'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
#        'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
#        'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
#        'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
#        'matchupPotentialAssists', 'matchupThreePointersAttempted',
#        'matchupThreePointersMade', 'matchupThreePointersPercentage',
#        'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
#        'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
#        'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']

In [82]:
off_matchup="Nikola Jokic"

In [83]:
fig_matchup_season_full = px.bar(
  matchup_season_full_totals[(matchup_season_full_totals[category_full_season]!=0)&(matchup_season_full_totals["MATCHUP_NAME"]==off_matchup)].sort_values(by=category_full_season, ascending=False).head(25),
  x="PLAYER_NAME", 
  y=category_full_season,
  color="PLAYER_NAME",
  color_discrete_map=player_color_dict_shot_dist,     
  template='plotly_white',
  title="<b>{} {} vs. Matchup For The {} Season</b>".format(off_matchup,category_full_season,year),
      hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints', 'SEASON'], 
  height=800, 
  width=1200,
  )
fig_matchup_season_full.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_matchup_season_full

In [ ]:
fig_matchup_season_full.write_html("Matchup {} For The {} Season".format(category_full_season,year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi Metric Plot

In [ ]:
category_full_season=['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']
for i in category_full_season:
       fig_matchup_season_full_multi = px.bar(
         matchup_season_full_totals[(matchup_season_full_totals[category_full_season]!=0)&(matchup_season_full_totals["MATCHUP_NAME"]==off_matchup)].sort_values(by=category_full_season, ascending=False).head(25),
       x="PLAYER_NAME", 
       y=category_full_season,
       color="PLAYER_NAME",
       color_discrete_map=player_color_dict_shot_dist,     
       template='plotly_white',
       title="<b>{} {} vs. Matchup For The {} Season</b>".format(off_matchup,category_full_season,year),
       hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints', 'SEASON'], 
       height=800, 
       width=1200,
       )
       fig_matchup_season_full_multi.update_layout(
       margin=dict(l=5, r=5, t=40, b=5),
       showlegend=False,
       title_x=0.5,
       hoverlabel=dict(
              bgcolor="white",
              font_size=16,
              font_family="PT Sans Narrow"
       ),
       yaxis = dict(tickfont = dict(size=15)),
       xaxis = dict(tickfont = dict(size=15)),
       font=dict(
              family="PT Sans Narrow",
              size=14,
              color="Black"
       ),
       title_font_family="PT Sans Narrow"
       )
       fig_matchup_season_full.write_html("Matchup {} For The {} Season".format(i,year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#For All Time 

In [49]:
#Combine player team data with player aggregated data so that we can color based on team
matchup_season_all=matchups_totals[(matchups_totals["Category"]=='Regular Season')]
matchup_season_all_totals=pd.DataFrame(matchup_season_all.groupby(by=['PLAYER_NAME','MATCHUP_NAME'])[['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
        'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']].sum()).reset_index()
matchup_season_all_totals["matchupFieldGoalsPercentage"]=matchup_season_all_totals["matchupFieldGoalsMade"]/matchup_season_all_totals["matchupFieldGoalsAttempted"]
matchup_season_all_totals["matchupThreePointersPercentage"]=matchup_season_all_totals["matchupThreePointersMade"]/matchup_season_all_totals["matchupThreePointersAttempted"]
matchup_season_all_totals["matchupFreeThrowsPercentage"]=matchup_season_all_totals["matchupFreeThrowsMade"]/matchup_season_all_totals["matchupFreeThrowsAttempted"]
matchup_season_all_totals=matchup_season_all_totals.fillna(0)

In [50]:
category_all="matchupFreeThrowsAttempted"

# category_all=['helpBlocks', 'helpFieldGoalsAttempted',
#        'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
#        'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
#        'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
#        'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
#        'matchupPotentialAssists', 'matchupThreePointersAttempted',
#        'matchupThreePointersMade', 'matchupThreePointersPercentage',
#        'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
#        'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
#        'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']

In [51]:
fig_matchup_all = px.bar(
  matchup_season_all_totals[(matchup_season_all_totals[category_all]!=0)&(matchup_season_all_totals["MATCHUP_NAME"]==off_matchup)].sort_values(by=category_all, ascending=False).head(25),
  x="PLAYER_NAME", 
  y=category_all,
  template='plotly_white',
  title="<b>{} Regular Season All Time {} vs. Matchup</b>".format(off_matchup, category_all),
  hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints'], 
  height=800, 
  width=1200,
  )
fig_matchup_all.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_matchup_all.update_traces(marker_color='#0047ab')
fig_matchup_all

In [ ]:
fig_matchup_all.write_html("Regular Season All Time Matchup {}".format(category_all),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi Plots

In [ ]:
category_all=['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']
for i in category_all:
       fig_matchup_all_multi = px.bar(
       matchup_season_all_totals[(matchup_season_all_totals[category_all]!=0)&(matchup_season_all_totals["MATCHUP_NAME"]==off_matchup)].sort_values(by=category_all, ascending=False).head(25),
       x="PLAYER_NAME", 
       y=category_all,
       template='plotly_white',
       title="<b>{} Regular Season All Time {} vs. Matchup</b>".format(off_matchup, category_all),
       hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints'], 
       height=800, 
       width=1200,
       )
       fig_matchup_all_multi.update_layout(
       margin=dict(l=5, r=5, t=40, b=5),
       showlegend=False,
       title_x=0.5,
       hoverlabel=dict(
              bgcolor="white",
              font_size=16,
              font_family="PT Sans Narrow"
       ),
       yaxis = dict(tickfont = dict(size=15)),
       xaxis = dict(tickfont = dict(size=15)),
       font=dict(
              family="PT Sans Narrow",
              size=14,
              color="Black"
       ),
       title_font_family="PT Sans Narrow"
       )
       fig_matchup_all_multi.update_traces(marker_color='#0047ab')
       fig_matchup_all_multi.write_html("Regular Season All Time Matchup {}".format(category_all),full_html=False, include_plotlyjs='cdn')

In [ ]:
#PLAYOFFS

In [ ]:
#TOTALS

In [34]:
year_playoffs="2023-24"

In [53]:
#Combine player team data with player aggregated data so that we can color based on team
matchup_totals_year_playoffs=matchups_totals[matchups_totals["Season"]==year].merge(player_team[player_team["SEASON"]==year], left_on=["MATCHUP_NAME","Season"], right_on=["MATCHUP_NAME","SEASON"])
matchup_totals_year_playoffs=matchup_totals_year_playoffs.merge(game_ids, left_on=["GAME_ID"], right_on=["GAME_ID"])
matchup_totals_year_playoffs[matchup_totals_year_playoffs["Category"]=="Playoffs"].tail()

,GAME_ID,HOME_TEAM,AWAY_TEAM,TEAM,PLAYER_NAME,POSITION,MATCHUP_NAME,helpBlocks,helpFieldGoalsAttempted,helpFieldGoalsMade,helpFieldGoalsPercentage,matchupAssists,matchupBlocks,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupFieldGoalsPercentage,matchupFreeThrowsAttempted,matchupFreeThrowsMade,matchupMinutes,matchupMinutesSort,matchupPotentialAssists,matchupThreePointersAttempted,matchupThreePointersMade,matchupThreePointersPercentage,matchupTurnovers,partialPossessions,percentageDefenderTotalTime,percentageOffensiveTotalTime,percentageTotalTimeBothOn,playerPoints,shootingFouls,switchesOn,teamPoints,Season,Category,TEAM_ABBREVIATION,SEASON,color,GAME_DATE,MATCHUP
218559,0042300203,Cleveland Cavaliers,Boston Celtics,Cleveland Cavaliers,Donovan Mitchell,G,Jrue Holiday,0,0,0,0.0,1,0,3,2,0.667,0,0,7:04,424,0,2,2,1.0,0,33.8,0.501,0.451,0.554,6,0,0,27,2023-24,Playoffs,BOS,2023-24,#007A33,2024-05-11,BOS @ CLE
218560,0042300163,Phoenix Suns,Minnesota Timberwolves,Phoenix Suns,Devin Booker,G,Mike Conley,0,0,0,0.0,2,0,3,2,0.667,2,2,1:30,89,0,0,0,0.0,2,9.6,0.131,0.111,0.151,6,0,0,10,2023-24,Playoffs,MIN,2023-24,#236192,2024-04-26,MIN @ PHX
218561,0042300236,Minnesota Timberwolves,Denver Nuggets,Minnesota Timberwolves,Jaden McDaniels,F,Jamal Murray,0,0,0,0.0,0,0,3,2,0.667,2,2,1:18,78,0,1,0,0.0,0,7.3,0.099,0.103,0.129,6,1,0,19,2023-24,Playoffs,DEN,2023-24,#1d428a,2024-05-16,DEN @ MIN
218562,0042300312,Minnesota Timberwolves,Dallas Mavericks,Dallas Mavericks,Kyrie Irving,G,Mike Conley,0,0,0,0.0,0,0,3,2,0.667,2,1,1:24,83,0,2,2,1.0,1,7.6,0.116,0.093,0.127,7,1,0,14,2023-24,Playoffs,MIN,2023-24,#236192,2024-05-24,DAL @ MIN
218563,0042300232,Denver Nuggets,Minnesota Timberwolves,Minnesota Timberwolves,Karl-Anthony Towns,C,Nikola Jokic,0,0,0,0.0,0,0,3,2,0.667,2,2,1:42,101,0,2,1,0.5,0,9.9,0.112,0.122,0.144,7,1,0,13,2023-24,Playoffs,DEN,2023-24,#1d428a,2024-05-06,MIN @ DEN


In [ ]:
#By Individual Games

In [54]:
game_id_playoffs="0042300232"

In [60]:
#Filter to regular season
matchup_season_playoffs=matchup_totals_year_playoffs[(matchup_totals_year_playoffs["Category"]=='Playoffs')&(matchup_totals_year_playoffs["GAME_ID"]==game_id_playoffs)]
matchup_season_playoffs.head()

,GAME_ID,HOME_TEAM,AWAY_TEAM,TEAM,PLAYER_NAME,POSITION,MATCHUP_NAME,helpBlocks,helpFieldGoalsAttempted,helpFieldGoalsMade,helpFieldGoalsPercentage,matchupAssists,matchupBlocks,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupFieldGoalsPercentage,matchupFreeThrowsAttempted,matchupFreeThrowsMade,matchupMinutes,matchupMinutesSort,matchupPotentialAssists,matchupThreePointersAttempted,matchupThreePointersMade,matchupThreePointersPercentage,matchupTurnovers,partialPossessions,percentageDefenderTotalTime,percentageOffensiveTotalTime,percentageTotalTimeBothOn,playerPoints,shootingFouls,switchesOn,teamPoints,Season,Category,TEAM_ABBREVIATION,SEASON,color,GAME_DATE,MATCHUP
391,0042300232,Denver Nuggets,Minnesota Timberwolves,Denver Nuggets,Kentavious Caldwell-Pope,G,Naz Reid,0,0,0,0.0,0,0,0,0,0.0,0,0,0:10,10,0,0,0,0.0,0,0.8,0.016,0.017,0.043,0,0,0,0,2023-24,Playoffs,MIN,2023-24,#236192,2024-05-06,MIN @ DEN
392,0042300232,Denver Nuggets,Minnesota Timberwolves,Denver Nuggets,Christian Braun,,Monte Morris,0,0,0,0.0,0,0,0,0,0.0,0,0,0:14,13,0,0,0,0.0,0,2.3,0.098,0.061,0.187,0,0,0,3,2023-24,Playoffs,MIN,2023-24,#236192,2024-05-06,MIN @ DEN
393,0042300232,Denver Nuggets,Minnesota Timberwolves,Denver Nuggets,Julian Strawther,,Josh Minott,0,0,0,0.0,0,0,0,0,0.0,0,0,0:05,4,0,0,0,0.0,0,0.5,0.065,0.057,0.065,0,0,0,0,2023-24,Playoffs,MIN,2023-24,#236192,2024-05-06,MIN @ DEN
394,0042300232,Denver Nuggets,Minnesota Timberwolves,Denver Nuggets,Zeke Nnaji,,Josh Minott,0,0,0,0.0,0,0,0,0,0.0,0,0,0:09,9,0,0,0,0.0,0,0.7,0.121,0.107,0.121,0,0,0,0,2023-24,Playoffs,MIN,2023-24,#236192,2024-05-06,MIN @ DEN
395,0042300232,Denver Nuggets,Minnesota Timberwolves,Denver Nuggets,Hunter Tyson,,Jordan McLaughlin,0,0,0,0.0,0,0,0,0,0.0,0,0,0:06,5,0,0,0,0.0,0,0.3,0.076,0.067,0.076,0,0,0,0,2023-24,Playoffs,MIN,2023-24,#236192,2024-05-06,MIN @ DEN


In [61]:
matchup_playoffs=matchup_season['MATCHUP'].iloc[0]
date_playoffs=matchup_season['GAME_DATE'].iloc[0]

In [62]:
category_playoffs="matchupFreeThrowsAttempted"

# category_playoffs=['helpBlocks', 'helpFieldGoalsAttempted',
#        'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
#        'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
#        'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
#        'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
#        'matchupPotentialAssists', 'matchupThreePointersAttempted',
#        'matchupThreePointersMade', 'matchupThreePointersPercentage',
#        'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
#        'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
#        'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']

In [63]:
fig_matchup_season_playoffs = px.bar(
  matchup_season_playoffs[matchup_season_playoffs[category_playoffs]!=0].sort_values(by=category_playoffs, ascending=False),
  x="MATCHUP_NAME", 
  y=category_playoffs,
  color="MATCHUP_NAME",
  color_discrete_map=player_color_dict_shot_dist,     
  template='plotly_white',
  title="<b>Matchup {} For The {} Game on {}</b>".format(category_playoffs,matchup_playoffs,date_playoffs),
  hover_data=['GAME_DATE','MATCHUP','TEAM', 'PLAYER_NAME',
       'MATCHUP_NAME', 'helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints',
       'Category', 'SEASON', ], 
  height=800, 
  width=1200,
  )
fig_matchup_season_playoffs.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_matchup_season_playoffs

In [ ]:
fig_matchup_season_playoffs.write_html("Matchup {} For The {} Game on {}".format(category_playoffs,matchup_playoffs,date_playoffs),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOT

In [ ]:
category_multi_playoffs=['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']
for i in category_multi_playoffs:
       fig_matchup_season_multi_playoffs = px.bar(
       matchup_season_playoffs[matchup_season_playoffs[i]!=0].sort_values(by=i, ascending=False),
       x="MATCHUP_NAME", 
       y=i,
       color="MATCHUP_NAME",
       color_discrete_map=player_color_dict_shot_dist,     
       template='plotly_white',
       title="<b>Matchup {} For The {} Game on {}</b>".format(i,matchup,date),
       hover_data=['GAME_DATE','MATCHUP','TEAM', 'PLAYER_NAME',
              'MATCHUP_NAME', 'helpBlocks', 'helpFieldGoalsAttempted',
              'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
              'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
              'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
              'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
              'matchupPotentialAssists', 'matchupThreePointersAttempted',
              'matchupThreePointersMade', 'matchupThreePointersPercentage',
              'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
              'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
              'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints',
              'Category', 'SEASON', ], 
       height=800, 
       width=1200,
       )
       fig_matchup_season_multi_playoffs.update_layout(
       margin=dict(l=5, r=5, t=40, b=5),
       showlegend=False,
       title_x=0.5,
       hoverlabel=dict(
              bgcolor="white",
              font_size=16,
              font_family="PT Sans Narrow"
       ),
       yaxis = dict(tickfont = dict(size=15)),
       xaxis = dict(tickfont = dict(size=15)),
       font=dict(
              family="PT Sans Narrow",
              size=14,
              color="Black"
       ),
       title_font_family="PT Sans Narrow"
       )
       fig_matchup_season_multi_playoffs
       fig_matchup_season_multi_playoffs.write_html("Matchup {} For The {} Game on {}".format(i,matchup,date),full_html=False, include_plotlyjs='cdn')

In [ ]:
# For Total Season

In [65]:
matchup_season_full_playoffs=matchup_totals_year_playoffs[(matchup_totals_year_playoffs["Category"]=='Playoffs')]

In [66]:
matchup_season_full_totals_playoffs=pd.DataFrame(matchup_season_full_playoffs.groupby(by=['PLAYER_NAME',"MATCHUP_NAME","SEASON"])[['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
        'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']].sum()).reset_index()
matchup_season_full_totals_playoffs["matchupFieldGoalsPercentage"]=matchup_season_full_totals_playoffs["matchupFieldGoalsMade"]/matchup_season_full_totals_playoffs["matchupFieldGoalsAttempted"]
matchup_season_full_totals_playoffs["matchupThreePointersPercentage"]=matchup_season_full_totals_playoffs["matchupThreePointersMade"]/matchup_season_full_totals_playoffs["matchupThreePointersAttempted"]
matchup_season_full_totals_playoffs["matchupFreeThrowsPercentage"]=matchup_season_full_totals_playoffs["matchupFreeThrowsMade"]/matchup_season_full_totals_playoffs["matchupFreeThrowsAttempted"]
matchup_season_full_totals_playoffs=matchup_season_full_totals_playoffs.fillna(0)

In [67]:
matchup_season_full_totals_playoffs.head()

,PLAYER_NAME,MATCHUP_NAME,SEASON,helpBlocks,helpFieldGoalsAttempted,helpFieldGoalsMade,matchupAssists,matchupBlocks,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupFreeThrowsAttempted,matchupFreeThrowsMade,matchupMinutesSort,matchupPotentialAssists,matchupThreePointersAttempted,matchupThreePointersMade,matchupTurnovers,partialPossessions,playerPoints,shootingFouls,switchesOn,teamPoints,matchupFieldGoalsPercentage,matchupThreePointersPercentage,matchupFreeThrowsPercentage
0,A.J. Lawson,Aaron Wiggins,2023-24,0,0,0,0,0,0,0,0,0,13,0,0,0,0,2.0,0,0,0,0,NaN,NaN,NaN
1,A.J. Lawson,Al Horford,2023-24,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0.2,0,0,0,0,NaN,NaN,NaN
2,A.J. Lawson,Amir Coffey,2023-24,0,0,0,0,0,0,0,0,0,38,0,0,0,0,1.4,0,0,0,0,NaN,NaN,NaN
3,A.J. Lawson,Bones Hyland,2023-24,0,0,0,0,0,1,1,0,0,48,0,1,1,0,3.1,3,0,0,10,1.0,1.0,NaN
4,A.J. Lawson,Brandon Boston Jr.,2023-24,0,0,0,0,0,3,1,0,0,80,0,2,0,0,6.9,2,0,0,7,0.333333,0.0,NaN


In [71]:
category_full_season_playoffs="playerPoints"

# category=['helpBlocks', 'helpFieldGoalsAttempted',
#        'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
#        'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
#        'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
#        'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
#        'matchupPotentialAssists', 'matchupThreePointersAttempted',
#        'matchupThreePointersMade', 'matchupThreePointersPercentage',
#        'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
#        'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
#        'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']

In [84]:
fig_matchup_season_full_playoffs = px.bar(
  matchup_season_full_totals_playoffs[(matchup_season_full_totals_playoffs[category_full_season_playoffs]!=0)&(matchup_season_full_totals_playoffs["MATCHUP_NAME"]==off_matchup)].sort_values(by=category_full_season_playoffs, ascending=False).head(25),
  x="PLAYER_NAME", 
  y=category_full_season_playoffs,
  color="PLAYER_NAME",
  color_discrete_map=player_color_dict_shot_dist,     
  template='plotly_white',
  title="<b>{} {} vs. Matchup For The {} Playoffs</b>".format(off_matchup, category_full_season_playoffs,year_playoffs),
  hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints', 'SEASON'], 
  height=800, 
  width=1200,
  )
fig_matchup_season_full_playoffs.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_matchup_season_full_playoffs

In [ ]:
fig_matchup_season_full_playoffs.write_html("Matchup {} For The {} Playoffs".format(category_full_season_playoffs,year_playoffs),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi Metric Plot

In [ ]:
category_full_season_playoffs=['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']
for i in category_full_season_playoffs:
       fig_matchup_season_full_multi_playoffs = px.bar(
       matchup_season_full_totals_playoffs[(matchup_season_full_totals_playoffs[category_full_season_playoffs]!=0)&(matchup_season_full_totals_playoffs["MATCHUP_NAME"]==off_matchup)].sort_values(by=category_full_season_playoffs, ascending=False).head(25),
       x="PLAYER_NAME", 
       y=category_full_season_playoffs,
       color="PLAYER_NAME",
       color_discrete_map=player_color_dict_shot_dist,     
       template='plotly_white',
         title="<b>{} {} vs. Matchup For The {} Playoffs</b>".format(off_matchup, category_full_season_playoffs,year_playoffs),
       hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints', 'SEASON'], 
       height=800, 
       width=1200,
       )
       fig_matchup_season_full_multi_playoffs.update_layout(
       margin=dict(l=5, r=5, t=40, b=5),
       showlegend=False,
       title_x=0.5,
       hoverlabel=dict(
              bgcolor="white",
              font_size=16,
              font_family="PT Sans Narrow"
       ),
       yaxis = dict(tickfont = dict(size=15)),
       xaxis = dict(tickfont = dict(size=15)),
       font=dict(
              family="PT Sans Narrow",
              size=14,
              color="Black"
       ),
       title_font_family="PT Sans Narrow"
       )
       fig_matchup_season_full_playoffs.write_html("Matchup {} For The {} Playoffs".format(i,year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#For All Time 

In [73]:
#Combine player team data with player aggregated data so that we can color based on team
matchup_season_all_playoffs=matchups_totals[(matchups_totals["Category"]=='Playoffs')]
matchup_season_all_totals_playoffs=pd.DataFrame(matchup_season_all_playoffs.groupby(by=['PLAYER_NAME',"MATCHUP_NAME"])[['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
        'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']].sum()).reset_index()
matchup_season_all_totals_playoffs["matchupFieldGoalsPercentage"]=matchup_season_all_totals_playoffs["matchupFieldGoalsMade"]/matchup_season_all_totals_playoffs["matchupFieldGoalsAttempted"]
matchup_season_all_totals_playoffs["matchupThreePointersPercentage"]=matchup_season_all_totals_playoffs["matchupThreePointersMade"]/matchup_season_all_totals_playoffs["matchupThreePointersAttempted"]
matchup_season_all_totals_playoffs["matchupFreeThrowsPercentage"]=matchup_season_all_totals_playoffs["matchupFreeThrowsMade"]/matchup_season_all_totals_playoffs["matchupFreeThrowsAttempted"]
matchup_season_all_totals_playoffs=matchup_season_all_totals_playoffs.fillna(0)

In [74]:
category_all_playoffs="playerPoints"

# category_all_playoffs=['helpBlocks', 'helpFieldGoalsAttempted',
#        'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
#        'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
#        'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
#        'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
#        'matchupPotentialAssists', 'matchupThreePointersAttempted',
#        'matchupThreePointersMade', 'matchupThreePointersPercentage',
#        'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
#        'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
#        'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']

In [76]:
fig_matchup_all_playoffs = px.bar(
  matchup_season_all_totals_playoffs[(matchup_season_all_totals_playoffs[category_all_playoffs]!=0)&(matchup_season_all_totals_playoffs["MATCHUP_NAME"]==off_matchup)].sort_values(by=category_all_playoffs, ascending=False).head(25),
  x="PLAYER_NAME", 
  y=category_all_playoffs,
  template='plotly_white',
  title="<b>{} Playoffs All Time {} vs. Matchup</b>".format(off_matchup, category_all_playoffs),
  hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints'], 
  height=800, 
  width=1200,
  )
fig_matchup_all_playoffs.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_matchup_all_playoffs.update_traces(marker_color='#0047ab')
fig_matchup_all_playoffs

In [ ]:
fig_matchup_all_playoffs.write_html("Playoffs All Time Matchup {}".format(category_all),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi Plots

In [ ]:
category_all_playoffs=['helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'helpFieldGoalsPercentage', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutes', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions', 'percentageDefenderTotalTime',
       'percentageOffensiveTotalTime', 'percentageTotalTimeBothOn',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints']
for i in category_all_playoffs:
       fig_matchup_all_multi_playoffs = px.bar(
       matchup_season_all_totals_playoffs[(matchup_season_all_totals_playoffs[category_all_playoffs]!=0)&(matchup_season_all_totals_playoffs["MATCHUP_NAME"]==off_matchup)].sort_values(by=category_all_playoffs, ascending=False).head(25),
       x="PLAYER_NAME", 
       y=category_all_playoffs,
       template='plotly_white',
       title="<b>{} Playoffs All Time {} vs. Matchup</b>".format(off_matchup, category_all_playoffs),
       hover_data=['PLAYER_NAME','helpBlocks', 'helpFieldGoalsAttempted',
       'helpFieldGoalsMade', 'matchupAssists',
       'matchupBlocks', 'matchupFieldGoalsAttempted', 'matchupFieldGoalsMade',
       'matchupFieldGoalsPercentage', 'matchupFreeThrowsAttempted',
       'matchupFreeThrowsMade', 'matchupMinutesSort',
       'matchupPotentialAssists', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupThreePointersPercentage',
       'matchupTurnovers', 'partialPossessions',
       'playerPoints', 'shootingFouls', 'switchesOn', 'teamPoints'], 
       height=800, 
       width=1200,
       )
       fig_matchup_all_multi_playoffs.update_layout(
       margin=dict(l=5, r=5, t=40, b=5),
       showlegend=False,
       title_x=0.5,
       hoverlabel=dict(
              bgcolor="white",
              font_size=16,
              font_family="PT Sans Narrow"
       ),
       yaxis = dict(tickfont = dict(size=15)),
       xaxis = dict(tickfont = dict(size=15)),
       font=dict(
              family="PT Sans Narrow",
              size=14,
              color="Black"
       ),
       title_font_family="PT Sans Narrow"
       )
       fig_matchup_all_multi_playoffs.update_traces(marker_color='#0047ab')
       fig_matchup_all_multi_playoffs.write_html("Playoffs All Time Matchup {}".format(category_all),full_html=False, include_plotlyjs='cdn')